<a href="https://colab.research.google.com/github/elpiprin/NLP-Stuff/blob/main/RNN_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

***Imports***

In [8]:
#https://www.tensorflow.org/text/tutorials/text_generation
import tensorflow as tf

import numpy as np
import os
import time

***Reading the data***

In [11]:
#Read, then decode for py2 compat.
text = open('/content/Articles.txt', 'rb').read().decode(encoding='utf-8')
#length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 480143 characters


In [12]:
#Take a look at the first 250 characters in text
print(text[:250])

Melissa Caddick: Death of Australian fraudster remains a mystery

When conwoman Melissa Caddick vanished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed foot found washed up on a beach months later - it 


In [ ]:
#The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

### **Processing the text**

***Vectorising the text***

In [16]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [17]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [18]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

In [19]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [20]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [21]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [22]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

***Prediction task*** 

***Creating training examples and targets*** 

In [23]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(480143,), dtype=int64, numpy=array([43, 63, 70, ..., 76, 77, 16])>

In [24]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [25]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

M
e
l
i
s
s
a
 
C
a


In [26]:
seq_length = 100

In [27]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'M' b'e' b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c'
 b'k' b':' b' ' b'D' b'e' b'a' b't' b'h' b' ' b'o' b'f' b' ' b'A' b'u'
 b's' b't' b'r' b'a' b'l' b'i' b'a' b'n' b' ' b'f' b'r' b'a' b'u' b'd'
 b's' b't' b'e' b'r' b' ' b'r' b'e' b'm' b'a' b'i' b'n' b's' b' ' b'a'
 b' ' b'm' b'y' b's' b't' b'e' b'r' b'y' b'\r' b'\n' b'\r' b'\n' b'W' b'h'
 b'e' b'n' b' ' b'c' b'o' b'n' b'w' b'o' b'm' b'a' b'n' b' ' b'M' b'e'
 b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c' b'k' b' '
 b'v' b'a' b'n'], shape=(101,), dtype=string)


In [28]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'
b'ished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed fo'
b'ot found washed up on a beach months later - it set off a frenzy in Australia.\r\n\r\nThe case blindsided'
b' investors, baffled police, and captured the imagination of a nation.\r\n\r\nThe fraudster has inspired a'
b' hit podcast, a TV dramatisation, and countless outlandish theories - including that she had been swa'


In [29]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [30]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [31]:
dataset = sequences.map(split_input_target)

In [32]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick va'
Target: b'elissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'


***Creating training batches***

In [33]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Building the model**

In [34]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [35]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [36]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### **Trying the model**

In [37]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 106) # (batch_size, sequence_length, vocab_size)


In [38]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  27136     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  108650    
                                                                 
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


In [39]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [40]:
sampled_indices

array([ 69,  60,  69,  52,  19,   2,  23,  68,  15,  37,  28,  31,   0,
         7,  34,  80,  70,  68,  95,  72,  67,  94,  79,  54,  29,  22,
        22, 104,  11,  60,  30,  90,  11,  75,  30,  85,  55, 101,  87,
        69,  39,  14,  21,  59,  30,  44,  86,   1,  28,  12,  70,  98,
        45,  63,  96,  21,   8,  24,  69,  65,  69,  14,  65,  62,  90,
         4,  10,  96,  20,  74,  76,  38,  90,  62,  53,  45,  56,  65,
        74,  25,  78,  19,   5,  53, 103,  22,   5,   8,  58,  41,  95,
         9,  28,  48,  16,  89,  34, 103,  88,  85])

In [41]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'or years.\xe2\x80\x9d\r\n\r\nA Boston native and long admirer of the Kennedy family, McIntyre said she was not both'

Next Char Predictions:
 b'kbkV1\r5j-G:A[UNK]$Dvlj\xc3\xbcni\xc3\xbauX;44\xe2\x80\xaf(b?\xc3\xad(q?\xc2\xa3Y\xe2\x80\x9c\xc3\xa3kI,3a?N\xc3\xa1\n:)l\xe2\x80\x94Oe\xe2\x80\x8b3%6kgk,gd\xc3\xad!\'\xe2\x80\x8b2prH\xc3\xaddWOZgp7t1"W\xe2\x80\xa64"%]K\xc3\xbc&:R.\xc3\xa9D\xe2\x80\xa6\xc3\xa8\xc2\xa3'


### **Training the model**

***Attaching an optimiser & a loss function***

In [42]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [43]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 106)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.663481, shape=(), dtype=float32)


In [44]:
tf.exp(example_batch_mean_loss).numpy()

106.00447

In [45]:
model.compile(optimizer='adam', loss=loss)

***Configuring checkpoints***

In [46]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

***Executing the training***

In [47]:
EPOCHS = 30

In [48]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
74/74 [==============================] - 482s 6s/step - loss: 3.1844
Epoch 2/30
74/74 [==============================] - 520s 7s/step - loss: 2.4041
Epoch 3/30
74/74 [==============================] - 561s 8s/step - loss: 2.2126
Epoch 4/30
74/74 [==============================] - 487s 7s/step - loss: 2.0555
Epoch 5/30
74/74 [==============================] - 482s 6s/step - loss: 1.9113
Epoch 6/30
74/74 [==============================] - 477s 6s/step - loss: 1.7727
Epoch 7/30
74/74 [==============================] - 501s 7s/step - loss: 1.6457
Epoch 8/30
74/74 [==============================] - 486s 7s/step - loss: 1.5384
Epoch 9/30
74/74 [==============================] - 485s 7s/step - loss: 1.4528
Epoch 10/30
74/74 [==============================] - 479s 6s/step - loss: 1.3817
Epoch 11/30
74/74 [==============================] - 482s 6s/step - loss: 1.3174
Epoch 12/30
74/74 [==============================] - 481s 6s/step - loss: 1.2588
Epoch 13/30
74/74 [==================

### **Generating Text**

In [49]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [50]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [52]:
#Text 1
start = time.time()
states = None
next_char = tf.constant(['Climate activists'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Climate activists to the police areas orgering in tendor attacks and I dividen Zones".

The Taliban have been opining out of water arrested after the scandal higher transport three chose in French and ISIS-K, not only by consect that their F-Kamp Can one babiles for a dozency that "Pessite that they were imposed for what’s taken aimit.

China’s various the division over from British to Cardiff Constitution providers who were stolen at births. The government now selling goes units site Police Scotland a famous institution usage. And his wod Unable to treat bubbles they brother's drought and heard”," says Mr Kade shrows having happened" agreed. Some stebtic

A 79-19 real rates may be all vistory. One is an unethical says. "It is a German ta essault's decision in nets are stucked under Hunda area, a famous classical musicians of Yehas Housing another user resources in car from having home to go enough.

Capcillut Rep. They explains, we took the athletes to to ensure where with a few hundr

In [54]:
#Text 2
start = time.time()
states = None
next_char = tf.constant(['Ukrainian refugees'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Ukrainian refugees crising in the Netherlands or swore charital at possible. "In see near the increasing scores the time on the number of skillingrorish militan and cemerinclusions on costle-Suld understand how tribunal relationships with the sprion. His wife is still unsure how he was to remain on students before it in Jurya, – when they work as personal elections in the right construction, she sought help, according to the streets have been trending out that he was a shason elte and everyone appeared to be security".

The first round of conservatives house in order before sent them offers spaces. The authorities are tinylighted by July Maybi, who is not on loos, including indued and hundreds of blood,” she said.

Propellig Pocalancy - they cannot all far another evidence online groups right at the about white prisonment, kithin his own experience with a tire, adding that her auto accused of smoke will never be used.

The group cases of waters.

With mifepristone was only teachers it 

In [55]:
#Text 3
start = time.time()
states = None
next_char = tf.constant(['Police investigations'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Police investigations and largest events and ending with a quip: the Willoughby to take us more repeated attacks,” a Météo France spokesman told CNN. “UNEP one of the influnning see the more preparing to offer those concernsibility to this help.

Harres on their villaged and Islam's two-thirdspolinians. This issue are presidential pills, they are going to feur by hervic show,” he said.

He saws in order to leh, a different find out there from year to eebs Russia or two attacks version of the virus, but two years older attempts to developing countries like Benin, where powerful behaviours and that it importable” on driffie. There was a story about the count single in his first way for higher. Oleh Lawson and Fane or LJJAMP, that schools had finally weeks about the FAA's edubabanity and now backstands by the Taliban’s capacity appear to be her first-near arrastads from the 301st and research group — when she considered some of the atsociate, whice makes it harder to keep their family. Me

In [57]:
#Text 4
start = time.time()
states = None
next_char = tf.constant(['Political rival'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Political rivals where the body sentence while its 18, process since 1960 in the Hull, the oming women and gar-passed organizations bying sold at and Europe im greets on both sides appear to be."

Pakistan shut down any net worries, this opying her family was first a person of colour.

Geoples asked if I I a cuturione? II affed the show's job failed a tribled off basers, that may not bann the run-of Delan wrote Corrying Been, a Pakistan politics, scientists this from open. The inquest was a stars. But the Valner Vanter also neeted some family and what they had worked in December. Any hope they might be all primate is constantly, Lawson* market, we court decades climate drime in Haryana's Khinh, will have taken on a much larger number of run business. We lost our business. All of this campaign has secured hourly Africa’

Wetter said he was finally cover, butinision in real targets, and are conducting more than 1,000 people with the Federal Airborne Roberbu, Explic Hissign told CNN that 

In [58]:
#Text 5
start = time.time()
states = None
next_char = tf.constant(['Pesidential candidate'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Pesidential candidate Roberu Man be learned from the strang, and corporate family, McCain and whether pre-subsequent accoult of his party. Medochat rewairs, however that his brother's customers it was a challenge from the wall!" says Dro Gole personnel the breaking platforms with a WhatsApp group, said Mr Hami.

Haw reached hyinggessions, had agreed to set off one on their none, says the number of ISIS-K recruits had down to quickly exuctable” that Bago restructions are grantfurally fighters by family in the tipen, the mobes yhark the use of nonps for severely immunocompromised processing many small times, but that they are given their lives. FMCansey Morteorology party (AeMs), chief of the 20th Cennery is a human rights organizations have promoded number of disagreement with his Spanish and India-ble.

Some research is a situation” of F-16s have been found in the accident.

He added that the pandemic still hakm to everyone's taste, though. One Lionestop of the Dead and the mority grou

In [59]:
#Text 6
start = time.time()
states = None
next_char = tf.constant(['Investment expert'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Investment expert with the industry, and her gettrace the general students. Jumber deported that Bhe Ittackards NaMP, Seale Durt, Chief of United States’ hurried deportation against him has brought in her life.

“She par forch, Kerising where he would us to like the couple outcome, former US Poston fell well is boes with eligible likely were taken against using this alleged data breach is being released by the right. What locas hurg to make a person of family money."

Roe varrit has already started a standard challenges. The scientists leading the Covid pandemic had become the poster boy for what behavioural insights exposed sense of more researchers at his house. She viewed markets is but we are so abortion". An Amiring Up Haition markets of the authorities’ refugees have been relyawed by the limits of two months after the vehal to competea GO One, including a blast in the bushes behaviour of rioters.

Police had arrived by the articles of interviver was front of missiles for sexual d

In [60]:
#Text 7
start = time.time()
states = None
next_char = tf.constant(['Controversy around'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Controversy around the content of at the accommodational strat high, US and Budge Bro knew moth, and he’s related to the World accusations details of their FAA antividual dueling or Visoproad Eatt Phill’s Providers Arnold in Boston.

After the security forces have been 1zmore and more reported in clurres, could be delay to replace learning up here.”

She’s concerns that from all cars than in North America. High plunge teams of propaganda that we want to look at home. 31 visas ground from a debase near the pregnancy.

The Ukrainian Brexit vaccination during Fake of Some Deputy Leslee, Operation (FPA), it first several devastating the broader car leader and shared with third-party common including the constait and battleal instructions intensity.

He educately has shown tear his first wife has punded to a recent trial, which was first probably fluenced a recest in a doesn’t know is in the region.”

“If you’re to furhy limining causing the far-right in Fifa, which has decaded his app.

Th

In [61]:
#Text 8
start = time.time()
states = None
next_char = tf.constant(['Anti-vaccine activist'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Anti-vaccine activist, Alwhat has accused of harroling's bedone and organisations will handle this has happened.

"More over the fear, so the) will creates that day, an his met on Hindu woman at the time, note CNO sel as minutes ground for the new areas, in both sides such as "less and a city, but the vice president at has cultred in 2020 landmains about the potential use of her relationship of the gun’s. The disticult capability that can be a half of victory in a strongly with them, felt like or displacting from one year of results for the opposition was published by Senause a recession and tells of pating something on Twitter.

So the bullies, I think there’s no deal with the immunocompromised adults. “We don’t know if we would change conduct its too protect those concessions to have countries a real eytablishment. The conspiracy theories are all riots were being information abortions in a stute where the early 1980s. But the point that it could massacred that recognise, however, tra

In [62]:
#Text 9
start = time.time()
states = None
next_char = tf.constant(['Abortion laws'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Abortion laws when it was seated asylum in the internet, campaign in the days after the peak of any 5000- with Few World Cup security advances. He says that the graveshing mines of her death will ultimately him to be used to catch the entire system," says Nvidia.

UN Buta Willoughby had received that her rather than killed while the Eir Faffishor of the government’s handling of Turkey.

At first thing they want to searching its and talk about the death night.”

“Nets are consurraised by the regional election was professional and more remented and trying to do say, abortion-edecated "companying platform in food patchy and professional complaints.

Before the Bell spoke with Westors again.

Some perspective research, with Wilburn told them about the PAC's role, identified the military, the ofDep was arrested Nixon alpo political maneral investment without requirements for only and police itself. Voting is convinced he had sent the opposition lead only in his on the floor next week agains

In [63]:
#Text 10
start = time.time()
states = None
next_char = tf.constant(['The scandal surrounding'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

The scandal surrounding her fett-end their financial adviser's licence agreement that does induce mucosal immunity and that he wout the bubless of the intelliced was muchaks obviously about his return to need abortions.

The scanners were families our risked back to the strategies on the way for overturned the backs on Mesissia, where debott sit. “You are looking at a distance of Afternative for lead to their allowed by a group of Hong Kong Peop and Theresay, says, and elderly been used in a provincla, who bushed the lobby inform makes the West score, another a penalty forces of need to be "bolier typy with we’re through the stress - an increase in big his finances.

The impeachment of Paxton follows air simple: ennievedent changes in scores occurred.

Still, it said.

Our air defense system with child

He has studied the societal sigional computer chips. For computer games, Nerban it has finally explored the tombstone and found it had got the ground during the 14th Amendment.

Stark S

In [64]:
#Text 11
start = time.time()
states = None
next_char = tf.constant(['Conservatives attacked'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Conservatives attacked with residunt of URea – and impressive in the later than a year.

Aronomic neighboughout the Office and Jemor additt AI as its classical think about the blowionameh and professional rished that product its hopes on their rate since May 2021 in a family.”

For now, there have been followed by a recoulting barreas investigation.”

After a therebory not just time, and consectivity bills that challenge."

It was just really have an abused driving in relative to be hunts."

Mr Koletti, he dealed scientists concerned that it would not be arguined the increasing school. It dodn't happened the suggestions about the might of only because he hasn’t the one footlonder comment. But WhatsApp - with hourly updates, the obdistone as well as Pakistan's of the scheme to the number in December and laiders. "He keaps such as health”, two warn against the odds of any kind of intions". He show used to tell him,” he added.

Many of Nvidia's companies for each elected or Senaty was amo

In [65]:
#Text 12
start = time.time()
states = None
next_char = tf.constant(['Liberals claimed'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Liberals claimed the border intensities to have conducted a strame with the US, Ukrainian pilots like Lieberman, an American when I didn't just thing in both cases against him has been "lived from Russian air-optabli. Bue prison, the rebust of non-public is an effective ways, he’t as part of the other history and John D. Khan of acting out of extrany and also actually know is now regularly returning to the changing scheme.

But in a strategies out about the challenges in Ely, Cardiff?

Five neighbourhood was assest why she shares up here – in one case up. Barisanis at the scheme will also runs trailing matchesty that is the use of not landmark for the survivors, have also alleged that they want. That is celebrated's officials is burstitutional watching for general of Republicans that we see ausonologists are pussinger than not a bady without awread. Now if ever gair to present.

One of the chiefings and verified by the child’s father.

And the articles say voters in Ukraine hates say a

In [66]:
#Text 13
start = time.time()
states = None
next_char = tf.constant(['Human rights'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Human rights over House about the faulbs of histery," he says.

Oher ewhen it would be a good year by Robert V. Khried CFNC Paxton to believe this allegations of why US costing he was forced out because of what's happening eeferending. “The year 28 hours after attention that can complete its took corrivors on why its website.

Bankers must abant shared by time for the Vol, programmed, abortion pills by mail will could be able to close the baby anything you do that additional elternatively opponents by long. Debattern Utical safe with Resuary beyond the Turnish commenting out the city, some said Europe or Abstobits in the UK and Wales" have maintederwards off hard-begrow many he’d used to despair at the moternal community.

"I just, thing Facebook experiences about the impact of the onling towards the riots, ordered hurical estate and every lot. Ot fear the videos mate allegedly called the popularity of remote woo letter is able to identify the securities from companies with the situati

### **Exporting the generator**

In [68]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [70]:
states = None
next_char = tf.constant(['The trial against'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

The trial against the budget be detached from coverage of Kenya – have prevelue training pigest races, and keywers have pateled over the past threats they've seen that medal, feterwing his views on vaccine backgrounds and supprised over the plasers with a fience of support or derision that he was more like quality" on accidents, in women doesn’t see rightward for the number of being one of Hindus was hoppened with the user’s IP address and despite this year.

In Act-stone has been finched a total of £557, 1957, while Kissinger gaphnound ullising hungreds internationally. “It’s mostly the economy cars that providing voters: his owh city.

Some people who were wearing on to ext analysis, as I was hard like because, “We are about their embolies as a public rejoving I can deliver.”

A role, the coroner said.

He salsot the surgical opportunity and two international election while prehibility of a text is more often leads to a political failure by a rickert. The government shared data would

### **Advanced: Customised training**